In [ ]:
!pip install xmltodict

전기자동차 보급 활성화 및 전기자동차 사용자를 위한 전국 공공 및 민간 충전기 상태 실시간 제공

* 충전소ID, 충전기ID, 상태정보

In [ ]:
import requests
import xmltodict
import pandas as pd
import requests
import time

import warnings

# FutureWarning 무시 설정
warnings.filterwarnings("ignore", category=FutureWarning)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# 지역코드정보
region_mapping = {
    '11': '서울특별시',
    '26': '부산광역시',
    '27': '대구광역시',
    '28': '인천광역시',
    '29': '광주광역시',
    '30': '대전광역시',
    '31': '울산광역시',
    '36': '세종특별자치시',
    '41': '경기도',
    '43': '충청북도',
    '44': '충청남도',
    '46': '전라남도',
    '47': '경상북도',
    '48': '경상남도',
    '50': '제주특별자치도',
    '51': '강원특별자치도',
    '52': '전북특별자치도'
}

zcode = list(region_mapping.keys())

In [ ]:
# api_key
key = '..'

# 전기자동차 충전소 상태(실시간?)
def get_charger_condtion(key,zcode):
  url = 'http://apis.data.go.kr/B552584/EvCharger/getChargerStatus'
  params ={'serviceKey' : key , 'pageNo' : '1', 'numOfRows' : '9999', 'period' : '10', 'zcode' : zcode ,'dataType' :'json'}
  response = requests.get(url, params=params)
  data = xmltodict.parse(response.content)
  return data

In [ ]:
condtion = get_charger_condtion(key,'11')
condtion

{'response': {'header': {'resultCode': '00',
   'resultMsg': 'NORMAL SERVICE.',
   'totalCount': '1716',
   'pageNo': '2',
   'numOfRows': '0'},
  'body': {'items': None}}}

전기자동차 보급 활성화 및 전기자동차 사용자를 위한 전국 공공 및 민간 충전기 운영현황 실시간 제공

* 충전소명, 위치, 상태정보, 운영시간, 운영기관명,충전기 용량

In [ ]:
# 충전소 위치
def get_charger_info(key, zcode):
    url = 'http://apis.data.go.kr/B552584/EvCharger/getChargerInfo'
    params ={'serviceKey' : key , 'pageNo' : '1', 'numOfRows' : '9999', 'zcode' : zcode }
    response = requests.get(url, params=params)
    data = xmltodict.parse(response.content)
    item = data['response']['body']['items']['item']
    data= pd.json_normalize(item)

    return data

def fetch_all_data(key, zcodes):
    all_data = []  # 데이터를 저장할 리스트

    for zcode in zcodes:
        print(f'Fetching data for zcode: {zcode}')
        data = get_charger_info(key, zcode)
        all_data.append(data)

        # 10초 대기
        time.sleep(10)

    # 모든 데이터를 하나의 DataFrame으로 결합
    combined_data = pd.concat(all_data, ignore_index=True)

    return combined_data

In [ ]:
data = fetch_all_data(key, zcode)


Fetching data for zcode: 11
Fetching data for zcode: 26
Fetching data for zcode: 27
Fetching data for zcode: 28
Fetching data for zcode: 29
Fetching data for zcode: 30
Fetching data for zcode: 31
Fetching data for zcode: 36
Fetching data for zcode: 41
Fetching data for zcode: 43
Fetching data for zcode: 44
Fetching data for zcode: 46
Fetching data for zcode: 47
Fetching data for zcode: 48
Fetching data for zcode: 50
Fetching data for zcode: 51
Fetching data for zcode: 52


In [ ]:
# match_datas 저장
csv_file_path = '/content/drive/MyDrive/Eelctric_data.csv'
data.to_csv(csv_file_path, index=False)

In [ ]:
data[(data['zcode'] == '41') & (data['addr'].str.contains('오산'))]


수집한 데이터로 뭘 할 수 있을까?

*